In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
class Net(nn.Module): 
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number 
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension 
        num_features = 1
        for s in size:
            num_features *= s 
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
# net.parameters()를 사용하여 정의된 신경망의 학습가능한 매개변수들을 확인할 수 있음 
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [9]:
# 다음의 임의의 32*32 입력을 가정함
# 참고로 크기가 다른 입력을 받을 때는 입력의 크기를 재조정하거나 신경망 수정함 
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.1152, -0.0818, -0.0358, -0.0790,  0.0333,  0.1116, -0.0667, -0.0337,
         -0.1185, -0.0625]], grad_fn=<AddmmBackward0>)


In [10]:
# 오류역전파를 통해 그레이디언트를 구하기 전에 모든 가중치의 그레이디언트 버퍼들을 초기화 
net.zero_grad()
out.backward(torch.randn(1, 10))

In [14]:
# 손실 함수 정의 및 임의의 값들에 대해서 오차 결과 확인
# nn 패키지는 많이 사용되는 손실함수들을 제공하며, 해당 예제는 단순한 MSE를 사용 
output = net(input)
target = torch.randn(10) # a dummy target, for example
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()
loss = criterion(output, target) 
print(loss)

tensor(1.9629, grad_fn=<MseLossBackward0>)


In [15]:
# 앞에 코드에서 언급한 것과 같이 오류 역전파하기 전, 그레이디언트를 초기화해야 함 # backward() 수행 후 어떤 변화가 있는지 확인하고, 초기화의 필요성을 확인함 
net.zero_grad() # zeroes the gradient buffers of all parameters
print('conv1.bias.grad before backward') 
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


In [17]:
loss.backward()
print('conv1.bias.grad after backward') 
print(net.conv1.bias.grad)

conv1.bias.grad after backward
tensor([ 0.0123,  0.0039, -0.0078, -0.0081, -0.0220,  0.0024])


In [19]:
# 스토캐스틱경사하강법((미래)가중치=(현재)가중치–학습률*그레이디언트)을이용하여가중치갱신하는코드는다음과같음 
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
# 하지만 위 구현 코드보다 실제, torch.optim에서 구현되는 SDG, Adam, RMSProp 등을 사용함 # 오류 역전파에서 최적화하는 방법을 보인 예제 코드
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)
# in your training loop:
optimizer.zero_grad() # zero the gradient buffers 
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # Does the update